# AI - Vacuum agent, empty program

Created (R. Basili, Nov. 2020)  
Modified (C.D. Hromei, Oct 2024)

## Exercises

1. Fill the program of the `Agent` in order to reproduce the behaviour of random moves.
2. Fill the program of the `Agent` in order to search for the closest dirty room, go there and `Suck`.  
    **Hint**: you should modify the `Perception` method of the Environment as well to make it Completely Observable.
3. Modify the `Environment` locations into a 2x2 matrix instead of a row.
4. Modify the `Execute action` method of the Environment in order to implement a probabilistic failure of the action chosen by the Agent. In that case, the action should fail and nothing happens!

In [1]:
! pip install Pillow numpy tk

You should consider upgrading via the '/home/francesco/.pyenv/versions/3.9.16/bin/python3.9 -m pip install --upgrade pip' command.


In [2]:
from tkinter import *
import random
from agents import *

loc_A, loc_B, loc_C, loc_D = (0, 0), (1, 0), (0, 1), (1, 1)  # The three locations for the Vacuum world

MyImage = None

class Gui(Environment):

    """This GUI environment has three locations, A, B and C. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status."""

    def __init__(self, root, height=400, width=520):
        super().__init__()
        self.status = {loc_A: 'Dirty',
                       loc_B: 'Dirty', 
                       loc_C: 'Clean'}
        self.root = root
        self.height = height
        self.width = width
        self.canvas = None
        self.buttons = []
        self.create_canvas()
        self.create_buttons()

    def thing_classes(self):
        """The list of things which can be used in the environment."""
        return [Wall, Dirt, ReflexVacuumAgent, RandomVacuumAgent,
                TableDrivenVacuumAgent, ModelBasedVacuumAgent]

    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        print('Perceived Location=', agent.location,' - Perceived StatusAtLoc=',  self.status[agent.location])
        return (agent.location, self.status) #(agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Change the location status (Dirty/Clean); track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        print('Executing action:', action)
        if action == 'Right':
            if agent.location == loc_A:
                agent.location = loc_B
            elif agent.location == loc_C:
                agent.location = loc_D
            agent.performance -= 1
        elif action == 'Left':
            if agent.location == loc_B:
                agent.location = loc_A
            elif agent.location == loc_D:
                agent.location = loc_C
            agent.performance -= 1
        elif action == 'Up':
            if agent.location == loc_C:
                agent.location = loc_A
            elif agent.location == loc_D:
                agent.location = loc_B
            agent.performance -= 1
        elif action == 'Down':
            if agent.location == loc_A:
                agent.location = loc_C
            elif agent.location == loc_B:
                agent.location = loc_D
            agent.performance -= 1
        
        elif action == 'Suck':
    
            if agent.location == loc_A:
                self.buttons[0].config(bg='white', activebackground='light grey')
            elif agent.location == loc_B:
                self.buttons[1].config(bg='white', activebackground='light grey')
            elif agent.location == loc_C:
                self.buttons[2].config(bg='white', activebackground='light grey')
            elif agent.location == loc_D:
                self.buttons[3].config(bg='white', activebackground='light grey')
            self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        """Agents start in either location at random."""
        return random.choice([loc_A, loc_B, loc_C, loc_D])

    ###################################################################################################
    #                        INTERFACE METHODS NOT USEFULL FOR THE AGENT/ENVIRONMENT
    ###################################################################################################
    def create_canvas(self):
        """Creates Canvas element in the GUI."""
        self.canvas = Canvas(
            self.root,
            width=self.width,
            height=self.height,
            background='powder blue')
        self.canvas.pack(side='bottom')

    def create_buttons(self):
        """Creates the buttons required in the GUI."""
        button_A = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_A.config(command=lambda btn=button_A: self.dirt_switch(btn))
        self.buttons.append(button_A)
        self.canvas.create_window(130, 200, anchor=N, window=button_A)

        button_B = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_B.config(command=lambda btn=button_B: self.dirt_switch(btn))
        self.buttons.append(button_B)
        self.canvas.create_window(250, 200, anchor=N, window=button_B)

        button_C = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_C.config(command=lambda btn=button_C: self.dirt_switch(btn))
        self.buttons.append(button_C)
        self.canvas.create_window(130, 300, anchor=N, window=button_C)

        button_D = Button(self.root, height=4, width=12, padx=2, pady=2, bg='white')
        button_D.config(command=lambda btn=button_D: self.dirt_switch(btn))
        self.buttons.append(button_D)
        self.canvas.create_window(250, 300, anchor=N, window=button_D)

    def dirt_switch(self, button):
        """Gives user the option to put dirt in any tile."""
        bg_color = button['bg']
        if bg_color == 'saddle brown':
            button.config(bg='white', activebackground='light grey')
        elif bg_color == 'white':
            button.config(bg='saddle brown', activebackground='light goldenrod')

    def read_env(self):
        """Reads the current state of the GUI."""
        for i, btn in enumerate(self.buttons):
            if i == 0:
                if btn['bg'] == 'white':
                    self.status[loc_A] = 'Clean'
                else:
                    self.status[loc_A] = 'Dirty'
            elif i == 1:
                if btn['bg'] == 'white':
                    self.status[loc_B] = 'Clean'
                else:
                    self.status[loc_B] = 'Dirty'
            elif i == 2:
                if btn['bg'] == 'white':
                    self.status[loc_C] = 'Clean'
                else:
                    self.status[loc_C] = 'Dirty'
            elif i == 3:
                if btn['bg'] == 'white':
                    self.status[loc_D] = 'Clean' 
                else:
                    self.status[loc_D] = 'Dirty'

    def update_env(self, agent):
        """Updates the GUI according to the agent's action."""
        self.read_env()
        # print(self.status)
        before_step = agent.location
        self.step()   #execute_actions
        print('Current Environment Status: ', self.status)
        print('Current Agent Location: ',agent.location)
        move_agent(self, agent, before_step)

###################################################################################################
#                                           THE AGENT
###################################################################################################

def ReflexVacuumAgent():
    """A reflex agent for the two-state vacuum environment. [Figure 2.8]
    >>> agent = ReflexVacuumAgent()
    >>> environment = TrivialVacuumEnvironment()
    >>> environment.add_thing(agent)
    >>> environment.run()
    >>> environment.status == {(1,0):'Clean' , (0,0) : 'Clean'}
    True
    """
    
    def program(percept):
        
        location, status = percept
        print('Perceived Status: ', percept)
        print("Status", status)
        
        x = float('inf')
        y = float('inf')
        Dirty = False
        for position in status.keys():
            if status[position] == "Dirty" and abs(position[0] - location[0]) < x:
                x = position[0]
                Dirty = True
            if status[position] == "Dirty" and abs(position[1] - location[1]) < y:
                Dirty = True
                y = position[1]
        objective = (x, y)

        if not Dirty:
            return "Stay"
        if status[location] == 'Dirty':
            return "Suck"
        elif objective[1] - location[1] > 0:
            return "Down"
        elif objective[1] - location[1] < 0:
            return "Up"
        else:
            if objective[0] - location[0] >= 0:
                return "Right"
            else:
                return "Left"
   

    return Agent(program)

###################################################################################################
#                            INTERFACE METHODS TO CREATE AND MOVE THE AGENT
###################################################################################################

def create_agent(env, agent):
    """Creates the agent in the GUI and is kept independent of the environment."""
    env.add_thing(agent)
    agent_position_map = {
        loc_A: (120, 140),
        loc_B: (240, 140),
        loc_C: (120, 240),
        loc_D: (240, 240)
    }
    env.text = env.canvas.create_text(agent_position_map[agent.location], font="Helvetica 10 bold italic", text="Agent")
    env.agent_logo = env.canvas.create_image(agent_position_map[agent.location][0] - 40, agent_position_map[agent.location][1] - 40, image=MyImage)

def move_agent(env, agent, before_step):
    """Moves the agent in the GUI when 'next' button is pressed."""
    if agent.location == before_step:
        return
    
    dx = (agent.location[0] - before_step[0]) * 120
    dy = (agent.location[1] - before_step[1]) * 100
    env.canvas.move(env.text, dx, dy)
    env.canvas.move(env.agent_logo, dx, dy)

# TODO: Add more agents to the environment.
# TODO: Expand the environment to XYEnvironment.

###################################################################################################
#                                           MAIN LOOP
###################################################################################################
def main():
    """The main function of the program."""
    root = Tk()
    root.title("Vacuum Environment")
    root.geometry("500x500")
    root.resizable(0, 0)
    frame = Frame(root, bg='black')
    
    next_button = Button(frame, text='Next', height=2, width=6, padx=2, pady=2)
    next_button.pack(side='left')
    frame.pack(side='bottom')
    
    env = Gui(root)
    agent = ReflexVacuumAgent()
    create_agent(env, agent)
    
    next_button.config(command=lambda: env.update_env(agent))
    
    root.mainloop()


if __name__ == "__main__":
    main()


Perceived Location= (0, 0)  - Perceived StatusAtLoc= Clean
Perceived Status:  ((0, 0), {(0, 0): 'Clean', (1, 0): 'Dirty', (0, 1): 'Dirty', (1, 1): 'Dirty'})
Status {(0, 0): 'Clean', (1, 0): 'Dirty', (0, 1): 'Dirty', (1, 1): 'Dirty'}
Executing action: Right
Current Environment Status:  {(0, 0): 'Clean', (1, 0): 'Dirty', (0, 1): 'Dirty', (1, 1): 'Dirty'}
Current Agent Location:  (1, 0)
Perceived Location= (1, 0)  - Perceived StatusAtLoc= Dirty
Perceived Status:  ((1, 0), {(0, 0): 'Clean', (1, 0): 'Dirty', (0, 1): 'Dirty', (1, 1): 'Dirty'})
Status {(0, 0): 'Clean', (1, 0): 'Dirty', (0, 1): 'Dirty', (1, 1): 'Dirty'}
Executing action: Suck
Current Environment Status:  {(0, 0): 'Clean', (1, 0): 'Clean', (0, 1): 'Dirty', (1, 1): 'Dirty'}
Current Agent Location:  (1, 0)
Perceived Location= (1, 0)  - Perceived StatusAtLoc= Clean
Perceived Status:  ((1, 0), {(0, 0): 'Clean', (1, 0): 'Clean', (0, 1): 'Dirty', (1, 1): 'Dirty'})
Status {(0, 0): 'Clean', (1, 0): 'Clean', (0, 1): 'Dirty', (1, 1): 'Di